## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Xuanhua,CN,40.6103,115.0447,71.44,68,22,5.17,few clouds
1,1,Riyadh,SA,24.6877,46.7219,102.34,9,19,10.54,few clouds
2,2,Craig,US,40.5153,-107.5464,67.95,30,0,5.75,clear sky
3,3,Kindu,CD,-2.9500,25.9500,80.67,57,87,3.00,overcast clouds
4,4,Kismayo,SO,-0.3582,42.5454,78.39,83,29,12.24,light rain


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please Enter the minimun temprature for vacation locations: "))
max_temp = float(input("Please Enter the maximum temprature for vacation locations: "))


Please Enter the minimun temprature for loctions: 75
Please Enter the maximum temprature for loctions: 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_location = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [9]:
# 4a. Determine if there are any empty rows.
preffered_location.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preffered_location.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kindu,CD,80.67,overcast clouds,-2.9500,25.9500,
4,Kismayo,SO,78.39,light rain,-0.3582,42.5454,
6,Lompoc,US,77.85,clear sky,34.6391,-120.4579,
11,Kavieng,PG,81.18,light rain,-2.5744,150.7967,
15,Santa Maria Del Oro,MX,82.29,few clouds,25.9333,-105.3667,
17,Poum,NC,78.15,overcast clouds,-20.2333,164.0167,
18,Lorengau,PG,81.70,broken clouds,-2.0226,147.2712,
22,Georgetown,MY,84.13,scattered clouds,5.4112,100.3354,
23,Vila Velha,BR,75.15,broken clouds,-20.3297,-40.2925,
25,Saint-Jean-Port-Joli,CA,76.87,broken clouds,47.2171,-70.2658,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in clean_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print1 = hotels["results"][0]["name"]
        print(f"Hotel Found : {print1}")
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel Found : Résidence Okanda
Hotel Found : Agan-Hotel
Hotel Found : Holiday Inn Express Lompoc, an IHG Hotel
Hotel Found : Nusa Island Retreat
Hotel Found : Hotel Palacio Del Sol
Hotel not found... skipping.
Hotel Found : Lorengau Harbourside Hotel
Hotel Found : Cititel Penang
Hotel Found : Hotel Vitória Palace
Hotel Found : Motel Blanche d'Haberville
Hotel Found : HOTEL ISABELA
Hotel Found : Abaco Beach Resort
Hotel Found : Hotel Kou-Bugny
Hotel Found : Hotel Mirage
Hotel Found : KOPKAR Hotel Bintang Sintuk
Hotel Found : MOUNTAINVIEW RESORT (Official)
Hotel Found : Finch Bay Galapagos Hotel
Hotel Found : Tour Eiffel Hotel Benin
Hotel Found : Hilton Kuching
Hotel Found : SanBiz Lodge
Hotel Found : Hampton Inn & Suites Jamestown
Hotel Found : base de Campismo LosPortales de Guane
Hotel Found : Osprey Beach Hotel
Hotel Found : Hotel Hetman
Hotel Found : Hotel SKTC Grand Palace
Hotel Found : Premier Inn Margate hotel
Hotel Found : Isles Sunset Lodge
Hotel Found : Resorts Resource Group 

Hotel Found : Hotel Posada San Jose
Hotel Found : Tanna Lodge
Hotel Found : Aracan Hotel
Hotel Found : Cossam Guest House
Hotel Found : Fengshui Residence
Hotel Found : Қалдыбайдың үйі 🏘️
Hotel Found : HS Hotsson Smart Acapulco
Hotel Found : pulpo
Hotel Found : Heritage Park Hotel
Hotel Found : Playa Bonita Resort
Hotel Found : Moon light Furnished Units
Hotel Found : Mandela hôtel
Hotel Found : Pin Drop Hotel
Hotel Found : 900 Boutique INN
Hotel Found : Taloa Heights
Hotel Found : The Country Lodge Hotel
Hotel Found : Dibbo areaمنطقة دبو
Hotel Found : Best Western Plus South Coast Inn
Hotel Found : Banjanay Hotel
Hotel Found : Bonthe Holiday Village
Hotel not found... skipping.
Hotel Found : OYO 598 Peony Hotel


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
fig = gmaps.figure(center=(30, 31.0), zoom_level=1.5)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))